In [1]:
import pandas as pd
import numpy as np
import nltk
import io
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from keras.preprocessing import text as keras_text, sequence as keras_seq
from keras.utils import np_utils
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import Dense, Activation, Convolution1D, MaxPooling1D, Dropout, Flatten
from keras.models import Sequential, save_model
from sklearn.model_selection import train_test_split
from keras.layers import Embedding
from sklearn.metrics import classification_report,confusion_matrix

Using TensorFlow backend.


In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\ANMOL
[nltk_data]     KESHARWANI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
Tokenizer=RegexpTokenizer(r"\w+[a-z]+")   ## only word
word=set(stopwords.words("english"))

In [4]:
data_new=pd.read_csv('english_dataset.tsv', sep = '\t', encoding="latin-1")

In [65]:
Column_Sequence=np.array(data_new["text"],dtype="str")

In [66]:
fresh=[]
for ix in Column_Sequence:
    sen=ix.lower()
    sen=Tokenizer.tokenize(sen)
    wordss=[(Word) for Word in sen if len(Word)>2 and Word not in word]
    sentance=" ".join(wordss)
    fresh.append(sentance)

In [68]:
tokenizer = keras_text.Tokenizer(char_level = False,lower=True) #
t=tokenizer.fit_on_texts(list(fresh))
list_tokenized_train = tokenizer.texts_to_sequences(fresh)
X_t = keras_seq.pad_sequences(list_tokenized_train, maxlen=300,padding="post") #maxlen padding 100

In [69]:
#load pre-trained embedding:

import io
import os, re, csv, math, codecs
from tqdm import tqdm
##
#####load embeddings
#print('loading word embeddings..300 dimention.')
#embeddings_index = {}
#f = codecs.open('wiki.simple.vec', encoding='utf-8')
#for line in tqdm(f):
#    values = line.rstrip().rsplit(' ')
#    word = values[0]
#    coefs = np.asarray(values[1:], dtype='float32')
#    embeddings_index[word] = coefs
#f.close()
#print('found %s word vectors' % len(embeddings_index))

#GLOVE_DIR = "C\:\Users\Abhi\Desktop\Abhinav 2nd imp\Implementation_text and image\2_class"
embeddings_index = {}
f = open('glove.6B.300d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

Total 400000 word vectors in Glove 6B 100d.


In [20]:
##### Printing words and index pair 
index=tokenizer.word_index
#print index


###Vocabulary size
vocab_size = len(index)
#print vocab_size

In [70]:

##################################################################################
##################################################################################

#####embedding matrix
#print('preparing embedding matrix...')
#MAX_NB_WORDS = 800000
#embed_dim = 300
#words_not_found = []
#nb_words = min(MAX_NB_WORDS, len(index))
#embedding_matrix = np.zeros((nb_words, embed_dim))
#for word, i in index.items():
#    if i >= nb_words:
#        continue
#    embedding_vector = embeddings_index.get(word)
#   if (embedding_vector is not None) and len(embedding_vector) > 0:
#       # words not found in embedding index will be all-zeros.
#        embedding_matrix[i] = embedding_vector
#    else:
#        words_not_found.append(word)
#print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

embedding_matrix = np.random.random((len(tokenizer.word_index) + 1, 300))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
embedding_layer = Embedding(len(tokenizer.word_index) + 1,300,weights=[embedding_matrix],input_length=300,trainable=True)


In [71]:
max(tokenizer.word_counts.values())

651

In [72]:
len(tokenizer.word_counts)

10094

In [76]:
X_train=X_t

In [77]:
size_embedding = max(list(tokenizer.word_index.values()))

In [78]:
X_train.shape

(5852, 300)

In [79]:
YY=data_new["task_1"]

In [80]:
YY

0       NOT
1       HOF
2       NOT
3       NOT
4       NOT
5       NOT
6       NOT
7       HOF
8       NOT
9       NOT
10      NOT
11      HOF
12      NOT
13      NOT
14      NOT
15      HOF
16      NOT
17      NOT
18      NOT
19      NOT
20      NOT
21      NOT
22      NOT
23      HOF
24      NOT
25      NOT
26      NOT
27      HOF
28      NOT
29      NOT
       ... 
5822    HOF
5823    NOT
5824    NOT
5825    NOT
5826    HOF
5827    NOT
5828    NOT
5829    HOF
5830    HOF
5831    HOF
5832    HOF
5833    HOF
5834    HOF
5835    HOF
5836    HOF
5837    HOF
5838    HOF
5839    HOF
5840    HOF
5841    HOF
5842    HOF
5843    HOF
5844    HOF
5845    HOF
5846    HOF
5847    HOF
5848    NOT
5849    HOF
5850    HOF
5851    HOF
Name: task_1, Length: 5852, dtype: object

In [81]:
#train['sentiment'] = train['sentiment'].map({'OAG': 2, 'CAG': 1,'NAG': 0})
YY = YY.map({'NOT':0, 'HOF': 1})

In [82]:
YY

0       0
1       1
2       0
3       0
4       0
5       0
6       0
7       1
8       0
9       0
10      0
11      1
12      0
13      0
14      0
15      1
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      1
24      0
25      0
26      0
27      1
28      0
29      0
       ..
5822    1
5823    0
5824    0
5825    0
5826    1
5827    0
5828    0
5829    1
5830    1
5831    1
5832    1
5833    1
5834    1
5835    1
5836    1
5837    1
5838    1
5839    1
5840    1
5841    1
5842    1
5843    1
5844    1
5845    1
5846    1
5847    1
5848    0
5849    1
5850    1
5851    1
Name: task_1, Length: 5852, dtype: int64

In [83]:
#Y_L=data_new["Comment label"]

# One-hot encode data
Y_train=np_utils.to_categorical(YY)
#uniqe_class=np.unique(All_class)

In [84]:
Y_train.shape

(5852, 2)

In [85]:
x_train, x_test, y_train, y_test = train_test_split(X_train,Y_train, test_size=0.20, random_state=42)

In [86]:
y_train.shape

(4681, 2)

In [87]:
x_train.shape

(4681, 300)

In [88]:
max_len=300

In [89]:
x_train

array([[ 447, 1260,   45, ...,    0,    0,    0],
       [3293, 7220, 7221, ...,    0,    0,    0],
       [  33,    2, 3447, ...,    0,    0,    0],
       ...,
       [2467, 1540, 9385, ...,    0,    0,    0],
       [ 431, 9578,  641, ...,    0,    0,    0],
       [ 197, 2591, 4530, ...,    0,    0,    0]])

In [90]:
model = Sequential()
model.add(Embedding(size_embedding+1,100,input_length=max_len))

model.add( Convolution1D(8, 4 ))
model.add( Activation('relu') )

model.add( Convolution1D(16,4) )

model.add( Activation('relu'))
model.add(Convolution1D(64,4))
model.add( MaxPooling1D( pool_size=(4) ) )
model.add(Activation("relu"))
model.add(Convolution1D(64,4))
model.add(Activation("relu"))

model.add( Flatten() )

model.add( Dense(2) )

model.add( Activation('sigmoid') )

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 300, 100)          1009500   
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 297, 8)            3208      
_________________________________________________________________
activation_6 (Activation)    (None, 297, 8)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 294, 16)           528       
_________________________________________________________________
activation_7 (Activation)    (None, 294, 16)           0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 291, 64)           4160      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 72, 64)            0         
__________

In [91]:
from keras.callbacks import ModelCheckpoint
filepath = "task1_cnn_glove_last.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor="val_acc", verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [92]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

In [93]:
hist=model.fit(x_train,y_train,epochs=100,batch_size=32,validation_data=(x_test, y_test),callbacks=callbacks_list)

Train on 4681 samples, validate on 1171 samples
Epoch 1/100
4681/4681 [==============================] - ETA: 4:49 - loss: 0.6935 - acc: 0.343 - ETA: 2:39 - loss: 0.6899 - acc: 0.500 - ETA: 1:53 - loss: 0.6892 - acc: 0.520 - ETA: 1:30 - loss: 0.6883 - acc: 0.531 - ETA: 1:14 - loss: 0.6780 - acc: 0.568 - ETA: 1:04 - loss: 0.6753 - acc: 0.578 - ETA: 56s - loss: 0.6794 - acc: 0.575 - ETA: 52s - loss: 0.6903 - acc: 0.56 - ETA: 49s - loss: 0.6795 - acc: 0.58 - ETA: 46s - loss: 0.6809 - acc: 0.58 - ETA: 43s - loss: 0.6779 - acc: 0.58 - ETA: 40s - loss: 0.6800 - acc: 0.58 - ETA: 38s - loss: 0.6826 - acc: 0.57 - ETA: 36s - loss: 0.6798 - acc: 0.58 - ETA: 34s - loss: 0.6822 - acc: 0.57 - ETA: 33s - loss: 0.6813 - acc: 0.58 - ETA: 32s - loss: 0.6824 - acc: 0.57 - ETA: 31s - loss: 0.6818 - acc: 0.57 - ETA: 30s - loss: 0.6827 - acc: 0.57 - ETA: 29s - loss: 0.6817 - acc: 0.57 - ETA: 28s - loss: 0.6808 - acc: 0.58 - ETA: 28s - loss: 0.6791 - acc: 0.59 - ETA: 27s - loss: 0.6790 - acc: 0.59 - ETA: 27s

4681/4681 [==============================] - ETA: 21s - loss: 0.4100 - acc: 0.90 - ETA: 18s - loss: 0.4326 - acc: 0.92 - ETA: 17s - loss: 0.4251 - acc: 0.91 - ETA: 17s - loss: 0.4523 - acc: 0.87 - ETA: 19s - loss: 0.4474 - acc: 0.85 - ETA: 19s - loss: 0.4530 - acc: 0.85 - ETA: 19s - loss: 0.4539 - acc: 0.85 - ETA: 19s - loss: 0.4412 - acc: 0.86 - ETA: 19s - loss: 0.4489 - acc: 0.84 - ETA: 20s - loss: 0.4494 - acc: 0.85 - ETA: 20s - loss: 0.4415 - acc: 0.85 - ETA: 20s - loss: 0.4361 - acc: 0.86 - ETA: 19s - loss: 0.4314 - acc: 0.86 - ETA: 19s - loss: 0.4298 - acc: 0.85 - ETA: 18s - loss: 0.4209 - acc: 0.86 - ETA: 18s - loss: 0.4148 - acc: 0.86 - ETA: 18s - loss: 0.4127 - acc: 0.86 - ETA: 18s - loss: 0.4134 - acc: 0.85 - ETA: 18s - loss: 0.4070 - acc: 0.86 - ETA: 18s - loss: 0.4080 - acc: 0.85 - ETA: 17s - loss: 0.4082 - acc: 0.85 - ETA: 17s - loss: 0.4069 - acc: 0.85 - ETA: 18s - loss: 0.4021 - acc: 0.85 - ETA: 17s - loss: 0.3979 - acc: 0.85 - ETA: 17s - loss: 0.4000 - acc: 0.85 - ETA: 

4681/4681 [==============================] - ETA: 16s - loss: 0.0611 - acc: 0.96 - ETA: 14s - loss: 0.0580 - acc: 0.98 - ETA: 14s - loss: 0.0622 - acc: 0.98 - ETA: 18s - loss: 0.0781 - acc: 0.98 - ETA: 21s - loss: 0.0735 - acc: 0.98 - ETA: 20s - loss: 0.0708 - acc: 0.98 - ETA: 19s - loss: 0.0681 - acc: 0.98 - ETA: 18s - loss: 0.0675 - acc: 0.98 - ETA: 17s - loss: 0.0636 - acc: 0.98 - ETA: 17s - loss: 0.0614 - acc: 0.98 - ETA: 16s - loss: 0.0592 - acc: 0.98 - ETA: 17s - loss: 0.0561 - acc: 0.98 - ETA: 17s - loss: 0.0594 - acc: 0.98 - ETA: 17s - loss: 0.0566 - acc: 0.98 - ETA: 17s - loss: 0.0549 - acc: 0.98 - ETA: 17s - loss: 0.0542 - acc: 0.98 - ETA: 17s - loss: 0.0521 - acc: 0.98 - ETA: 17s - loss: 0.0616 - acc: 0.98 - ETA: 18s - loss: 0.0624 - acc: 0.98 - ETA: 17s - loss: 0.0612 - acc: 0.98 - ETA: 17s - loss: 0.0604 - acc: 0.98 - ETA: 17s - loss: 0.0589 - acc: 0.98 - ETA: 16s - loss: 0.0573 - acc: 0.98 - ETA: 16s - loss: 0.0556 - acc: 0.98 - ETA: 16s - loss: 0.0594 - acc: 0.98 - ETA: 

4681/4681 [==============================] - ETA: 17s - loss: 0.1149 - acc: 0.96 - ETA: 18s - loss: 0.0802 - acc: 0.96 - ETA: 21s - loss: 0.0707 - acc: 0.96 - ETA: 22s - loss: 0.0648 - acc: 0.96 - ETA: 21s - loss: 0.0862 - acc: 0.96 - ETA: 20s - loss: 0.0816 - acc: 0.96 - ETA: 19s - loss: 0.0716 - acc: 0.96 - ETA: 18s - loss: 0.0634 - acc: 0.97 - ETA: 18s - loss: 0.0593 - acc: 0.97 - ETA: 18s - loss: 0.0539 - acc: 0.97 - ETA: 18s - loss: 0.0499 - acc: 0.98 - ETA: 18s - loss: 0.0529 - acc: 0.97 - ETA: 17s - loss: 0.0496 - acc: 0.98 - ETA: 17s - loss: 0.0649 - acc: 0.97 - ETA: 17s - loss: 0.0610 - acc: 0.97 - ETA: 18s - loss: 0.0581 - acc: 0.98 - ETA: 18s - loss: 0.0567 - acc: 0.98 - ETA: 17s - loss: 0.0540 - acc: 0.98 - ETA: 17s - loss: 0.0515 - acc: 0.98 - ETA: 17s - loss: 0.0560 - acc: 0.98 - ETA: 16s - loss: 0.0538 - acc: 0.98 - ETA: 16s - loss: 0.0576 - acc: 0.98 - ETA: 16s - loss: 0.0558 - acc: 0.98 - ETA: 16s - loss: 0.0539 - acc: 0.98 - ETA: 16s - loss: 0.0540 - acc: 0.98 - ETA: 

4681/4681 [==============================] - ETA: 28s - loss: 0.0070 - acc: 1.00 - ETA: 24s - loss: 0.0073 - acc: 1.00 - ETA: 20s - loss: 0.0246 - acc: 0.98 - ETA: 19s - loss: 0.0282 - acc: 0.98 - ETA: 20s - loss: 0.0238 - acc: 0.98 - ETA: 20s - loss: 0.0219 - acc: 0.98 - ETA: 22s - loss: 0.0379 - acc: 0.98 - ETA: 21s - loss: 0.0338 - acc: 0.98 - ETA: 21s - loss: 0.0303 - acc: 0.98 - ETA: 20s - loss: 0.0275 - acc: 0.99 - ETA: 20s - loss: 0.0254 - acc: 0.99 - ETA: 21s - loss: 0.0237 - acc: 0.99 - ETA: 21s - loss: 0.0225 - acc: 0.99 - ETA: 21s - loss: 0.0211 - acc: 0.99 - ETA: 20s - loss: 0.0200 - acc: 0.99 - ETA: 20s - loss: 0.0189 - acc: 0.99 - ETA: 20s - loss: 0.0194 - acc: 0.99 - ETA: 20s - loss: 0.0189 - acc: 0.99 - ETA: 20s - loss: 0.0183 - acc: 0.99 - ETA: 20s - loss: 0.0204 - acc: 0.99 - ETA: 19s - loss: 0.0229 - acc: 0.99 - ETA: 19s - loss: 0.0225 - acc: 0.99 - ETA: 19s - loss: 0.0216 - acc: 0.99 - ETA: 19s - loss: 0.0209 - acc: 0.99 - ETA: 19s - loss: 0.0204 - acc: 0.99 - ETA: 

...

4681/4681 [==============================] - ETA: 17s - loss: 4.1688e-06 - acc: 1.00 - ETA: 19s - loss: 0.0103 - acc: 1.0000   - ETA: 20s - loss: 0.0068 - acc: 1.00 - ETA: 19s - loss: 0.0051 - acc: 1.00 - ETA: 18s - loss: 0.0041 - acc: 1.00 - ETA: 17s - loss: 0.0068 - acc: 1.00 - ETA: 17s - loss: 0.0089 - acc: 0.99 - ETA: 18s - loss: 0.0078 - acc: 0.99 - ETA: 18s - loss: 0.0092 - acc: 0.99 - ETA: 18s - loss: 0.0083 - acc: 0.99 - ETA: 17s - loss: 0.0075 - acc: 0.99 - ETA: 17s - loss: 0.0086 - acc: 0.99 - ETA: 16s - loss: 0.0113 - acc: 0.99 - ETA: 16s - loss: 0.0114 - acc: 0.99 - ETA: 16s - loss: 0.0122 - acc: 0.99 - ETA: 15s - loss: 0.0114 - acc: 0.99 - ETA: 16s - loss: 0.0121 - acc: 0.99 - ETA: 16s - loss: 0.0130 - acc: 0.99 - ETA: 15s - loss: 0.0127 - acc: 0.99 - ETA: 15s - loss: 0.0137 - acc: 0.99 - ETA: 15s - loss: 0.0131 - acc: 0.99 - ETA: 15s - loss: 0.0125 - acc: 0.99 - ETA: 16s - loss: 0.0119 - acc: 0.99 - ETA: 15s - loss: 0.0114 - acc: 0.99 - ETA: 15s - loss: 0.0110 - acc: 0.99

4681/4681 [==============================] - ETA: 13s - loss: 5.3050e-06 - acc: 1.00 - ETA: 18s - loss: 3.4236e-06 - acc: 1.00 - ETA: 18s - loss: 2.5854e-06 - acc: 1.00 - ETA: 19s - loss: 4.7128e-06 - acc: 1.00 - ETA: 18s - loss: 3.9304e-06 - acc: 1.00 - ETA: 17s - loss: 4.2042e-06 - acc: 1.00 - ETA: 16s - loss: 3.6526e-06 - acc: 1.00 - ETA: 17s - loss: 0.0029 - acc: 0.9961   - ETA: 17s - loss: 0.0026 - acc: 0.99 - ETA: 18s - loss: 0.0023 - acc: 0.99 - ETA: 18s - loss: 0.0021 - acc: 0.99 - ETA: 17s - loss: 0.0019 - acc: 0.99 - ETA: 16s - loss: 0.0018 - acc: 0.99 - ETA: 16s - loss: 0.0017 - acc: 0.99 - ETA: 16s - loss: 0.0016 - acc: 0.99 - ETA: 16s - loss: 0.0028 - acc: 0.99 - ETA: 17s - loss: 0.0039 - acc: 0.99 - ETA: 16s - loss: 0.0091 - acc: 0.99 - ETA: 16s - loss: 0.0086 - acc: 0.99 - ETA: 16s - loss: 0.0097 - acc: 0.99 - ETA: 16s - loss: 0.0114 - acc: 0.98 - ETA: 16s - loss: 0.0130 - acc: 0.98 - ETA: 16s - loss: 0.0124 - acc: 0.98 - ETA: 16s - loss: 0.0119 - acc: 0.98 - ETA: 15s - 

4681/4681 [==============================] - ETA: 17s - loss: 4.0971e-05 - acc: 1.00 - ETA: 21s - loss: 2.0775e-05 - acc: 1.00 - ETA: 22s - loss: 1.4120e-05 - acc: 1.00 - ETA: 21s - loss: 1.0783e-05 - acc: 1.00 - ETA: 19s - loss: 0.0080 - acc: 1.0000   - ETA: 18s - loss: 0.0103 - acc: 0.99 - ETA: 17s - loss: 0.0117 - acc: 0.99 - ETA: 16s - loss: 0.0103 - acc: 0.99 - ETA: 17s - loss: 0.0091 - acc: 0.99 - ETA: 17s - loss: 0.0082 - acc: 0.99 - ETA: 18s - loss: 0.0093 - acc: 0.99 - ETA: 17s - loss: 0.0085 - acc: 0.99 - ETA: 17s - loss: 0.0079 - acc: 0.99 - ETA: 17s - loss: 0.0090 - acc: 0.99 - ETA: 17s - loss: 0.0084 - acc: 0.99 - ETA: 17s - loss: 0.0078 - acc: 0.99 - ETA: 17s - loss: 0.0086 - acc: 0.99 - ETA: 16s - loss: 0.0098 - acc: 0.99 - ETA: 16s - loss: 0.0093 - acc: 0.99 - ETA: 16s - loss: 0.0092 - acc: 0.99 - ETA: 16s - loss: 0.0095 - acc: 0.99 - ETA: 16s - loss: 0.0101 - acc: 0.99 - ETA: 16s - loss: 0.0103 - acc: 0.99 - ETA: 16s - loss: 0.0099 - acc: 0.99 - ETA: 15s - loss: 0.0095

4681/4681 [==============================] - ETA: 16s - loss: 0.0016 - acc: 1.00 - ETA: 21s - loss: 0.0124 - acc: 0.98 - ETA: 21s - loss: 0.0086 - acc: 0.98 - ETA: 21s - loss: 0.0065 - acc: 0.99 - ETA: 21s - loss: 0.0053 - acc: 0.99 - ETA: 20s - loss: 0.0139 - acc: 0.98 - ETA: 20s - loss: 0.0119 - acc: 0.98 - ETA: 22s - loss: 0.0105 - acc: 0.98 - ETA: 21s - loss: 0.0135 - acc: 0.98 - ETA: 20s - loss: 0.0121 - acc: 0.98 - ETA: 20s - loss: 0.0110 - acc: 0.98 - ETA: 19s - loss: 0.0102 - acc: 0.98 - ETA: 19s - loss: 0.0094 - acc: 0.99 - ETA: 19s - loss: 0.0088 - acc: 0.99 - ETA: 19s - loss: 0.0102 - acc: 0.98 - ETA: 19s - loss: 0.0108 - acc: 0.99 - ETA: 19s - loss: 0.0102 - acc: 0.99 - ETA: 18s - loss: 0.0121 - acc: 0.98 - ETA: 18s - loss: 0.0127 - acc: 0.98 - ETA: 19s - loss: 0.0121 - acc: 0.98 - ETA: 19s - loss: 0.0115 - acc: 0.98 - ETA: 18s - loss: 0.0132 - acc: 0.98 - ETA: 18s - loss: 0.0137 - acc: 0.98 - ETA: 17s - loss: 0.0132 - acc: 0.98 - ETA: 17s - loss: 0.0126 - acc: 0.98 - ETA: 

4681/4681 [==============================] - ETA: 12s - loss: 8.3886e-05 - acc: 1.00 - ETA: 13s - loss: 3.4664e-04 - acc: 1.00 - ETA: 14s - loss: 0.0078 - acc: 0.9896   - ETA: 16s - loss: 0.0146 - acc: 0.99 - ETA: 17s - loss: 0.0135 - acc: 0.99 - ETA: 17s - loss: 0.0136 - acc: 0.99 - ETA: 16s - loss: 0.0117 - acc: 0.99 - ETA: 16s - loss: 0.0105 - acc: 0.99 - ETA: 16s - loss: 0.0219 - acc: 0.98 - ETA: 16s - loss: 0.0198 - acc: 0.98 - ETA: 17s - loss: 0.0201 - acc: 0.98 - ETA: 17s - loss: 0.0184 - acc: 0.98 - ETA: 16s - loss: 0.0182 - acc: 0.98 - ETA: 16s - loss: 0.0169 - acc: 0.98 - ETA: 15s - loss: 0.0172 - acc: 0.98 - ETA: 16s - loss: 0.0180 - acc: 0.99 - ETA: 16s - loss: 0.0169 - acc: 0.99 - ETA: 16s - loss: 0.0161 - acc: 0.99 - ETA: 16s - loss: 0.0164 - acc: 0.99 - ETA: 16s - loss: 0.0205 - acc: 0.98 - ETA: 16s - loss: 0.0206 - acc: 0.98 - ETA: 16s - loss: 0.0214 - acc: 0.98 - ETA: 16s - loss: 0.0215 - acc: 0.98 - ETA: 16s - loss: 0.0206 - acc: 0.98 - ETA: 16s - loss: 0.0207 - acc: 

In [94]:
from keras.models import load_model
# load weights into new model
loaded_model= load_model("task1_cnn_glove_last.hdf5")

In [95]:
w=model.predict_classes(x_test)

In [96]:
from sklearn.metrics import classification_report,confusion_matrix

In [97]:
Y_actual=[]
for ix in y_test:
    Y_actual.append(np.argmax(ix))

In [98]:
print(classification_report(Y_actual,w))

             precision    recall  f1-score   support

          0       0.67      0.59      0.63       734
          1       0.42      0.50      0.46       437

avg / total       0.57      0.56      0.56      1171



In [99]:
print(confusion_matrix(Y_actual,w))

[[435 299]
 [219 218]]


In [100]:
data_test=pd.read_csv('hasoc2019_en_test.tsv', sep = '\t', encoding="latin-1")

In [101]:
data_test.head()

,text_id,text
0,hasoc_en_902,West Bengal Doctor Crisis: Protesting doctors ...
1,hasoc_en_416,68.5 million people have been forced to leave ...
2,hasoc_en_207,"You came, you saw .... we will look after the ..."
3,hasoc_en_595,We'll get Brexit delivered by October 31st. ...
4,hasoc_en_568,Fuck you. Go back to the dark ages you cow @IB...


In [102]:
Column_Sequence=np.array(data_test["text"],dtype="str")
fresh=[]
for ix in Column_Sequence:
    sen=ix.lower()
    sen=Tokenizer.tokenize(sen)
    wordss=[(Word) for Word in sen if len(Word)>2 and Word not in word]
    sentance=" ".join(wordss)
    fresh.append(sentance)

In [103]:
tokenizer = keras_text.Tokenizer(char_level = False,lower=True) #
t=tokenizer.fit_on_texts(list(fresh))
list_tokenized_train = tokenizer.texts_to_sequences(fresh)
X_t = keras_seq.pad_sequences(list_tokenized_train, maxlen=300,padding="post") #maxlen padding 100

In [104]:
w_test=model.predict_classes(X_t)

In [105]:
w_test

array([1, 1, 0, ..., 0, 1, 1], dtype=int64)

In [106]:
print(len(w_test)-np.count_nonzero(w_test))
np.count_nonzero(w_test)

533


620

In [107]:
output = pd.DataFrame()
#out_result['text_id'] = 

In [108]:
output['text_id'] = data_test['text_id']

In [109]:
output.tail()

,text_id
1148,hasoc_en1_3958
1149,hasoc_en1_4648
1150,hasoc_en1_4832
1151,hasoc_en1_3721
1152,hasoc_en1_991


In [110]:
ans_dict = {0:'NOT',1:'HOF'}
w2 = np.vectorize(ans_dict.get)(w_test)
output['result'] = w2

In [111]:
output.to_csv('new_Kirti Kumari_English_task_1_run_2.tsv',header=True,sep='\t',index=False)